# 🏃 Real-Time Injury Risk Prediction — VS Code Edition

**Fully compatible with VS Code + Jupyter. No Google Colab dependencies.**

| Mode | Cell to Run |
|------|-------------|
| 🎥 Live Webcam (pops up a window) | Cell 6 |
| 📁 Video File | Cell 7 |
| 🌐 Streamlit Web App | Cell 8 |
| 📊 Summary Report | Cell 9 |

### Risk Factors Analyzed
| Factor | Joints | Risk Threshold |
|--------|--------|----------------|
| ACL / Knee Valgus | Hip → Knee → Ankle | < 160° |
| Hip Flexion | Shoulder → Hip → Knee | < 150° |
| Trunk Lean | Shoulder mid → Hip mid | > 15° from vertical |
| Shoulder Asymmetry | L vs R shoulder height | > 15px diff |
| Hip Asymmetry | L vs R hip height | > 15px diff |
| Ankle Dorsiflexion | Knee → Ankle → virtual toe | > 110° |

## Cell 1 — Install Dependencies

In [1]:
import subprocess, sys

packages = [
    'ultralytics',
    'opencv-python',
    'yt-dlp',
    'matplotlib',
    'numpy',
    'streamlit'
]

for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '--quiet'])
    print(f'✅ {pkg}')

print('\n🎉 All packages installed!')

✅ ultralytics
✅ opencv-python
✅ yt-dlp
✅ matplotlib
✅ numpy
✅ streamlit

🎉 All packages installed!


## Cell 2 — Configuration
**Edit these values before running anything else.**

In [4]:
# ─────────────────────────────────────────────────────────────────
#   ✏️  EDIT THESE VALUES
# ─────────────────────────────────────────────────────────────────

VIDEO_PATH      = 'sample.mp4'        # Path to your video file
OUTPUT_PATH     = 'output_risk.mp4'   # Where to save annotated video
MAX_FRAMES      = None                # None = all frames, or set a number e.g. 300

MODEL_SIZE      = 'n'                 # 'n'=fastest, 's'=balanced, 'm'=most accurate
CONF_THRESH     = 0.5                 # Keypoint confidence (0.0–1.0)

SHOW_ANGLES     = True                # Draw joint angle values on frame
SHOW_GRAPH      = True                # Show rolling risk history sparkline
FRAME_SKIP      = 1                   # Process every Nth frame (increase if slow)

ALERT_THRESHOLD = 70                  # Risk % that triggers a red alert flash
ALERT_COOLDOWN  = 3.0                 # Seconds between alerts

# Output folder for saved files (no google.colab needed)
import os
OUTPUT_FOLDER = os.path.join(os.path.expanduser('~'), 'Desktop', 'injury_risk_outputs')
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

print('✅ Configuration loaded')
print(f'   Output folder: {OUTPUT_FOLDER}')

✅ Configuration loaded
   Output folder: /Users/namantulsyan/Desktop/injury_risk_outputs


## Cell 3 — Imports & Model

In [5]:
import cv2
import numpy as np
import math
import time
import threading
import queue
import shutil
import warnings
from collections import deque
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from ultralytics import YOLO
warnings.filterwarnings('ignore')

model = YOLO(f'yolov8{MODEL_SIZE}-pose.pt')
print(f'✅ YOLOv8{MODEL_SIZE.upper()}-Pose loaded')

# Shared state
all_risk_scores = []
risk_history    = deque(maxlen=200)

✅ YOLOv8N-Pose loaded


## Cell 4 — Biomechanical Analysis Engine

In [6]:
# YOLOv8 Pose Keypoint Index Reference:
# 0=nose  1=l_eye  2=r_eye  3=l_ear  4=r_ear
# 5=l_shoulder  6=r_shoulder  7=l_elbow  8=r_elbow
# 9=l_wrist  10=r_wrist  11=l_hip  12=r_hip
# 13=l_knee  14=r_knee  15=l_ankle  16=r_ankle

def calculate_angle(a, b, c):
    """Angle at joint B formed by A-B-C (degrees)."""
    a, b, c = np.array(a, float), np.array(b, float), np.array(c, float)
    ba, bc = a - b, c - b
    n_ba, n_bc = np.linalg.norm(ba), np.linalg.norm(bc)
    if n_ba == 0 or n_bc == 0:
        return 180.0
    return float(np.degrees(np.arccos(np.clip(np.dot(ba, bc) / (n_ba * n_bc), -1.0, 1.0))))


def trunk_lean_angle(shoulder_mid, hip_mid):
    dx = hip_mid[0] - shoulder_mid[0]
    dy = hip_mid[1] - shoulder_mid[1]
    return 0.0 if dy == 0 else abs(math.degrees(math.atan2(dx, dy)))


def analyze_person(person_kp, person_conf):
    risks   = {}
    details = {}

    def kp(idx):
        return person_kp[idx] if (
            person_conf[idx] > CONF_THRESH and
            person_kp[idx][0] > 0 and person_kp[idx][1] > 0
        ) else None

    ls, rs = kp(5),  kp(6)
    lh, rh = kp(11), kp(12)
    lk, rk = kp(13), kp(14)
    la, ra = kp(15), kp(16)

    # Left knee (ACL / valgus)
    if all(v is not None for v in [lh, lk, la]):
        a = calculate_angle(lh, lk, la)
        details['L_Knee'] = (lk, a)
        risks['L_ACL'] = 90 if a < 120 else 60 if a < 140 else 30 if a < 160 else 0

    # Right knee
    if all(v is not None for v in [rh, rk, ra]):
        a = calculate_angle(rh, rk, ra)
        details['R_Knee'] = (rk, a)
        risks['R_ACL'] = 90 if a < 120 else 60 if a < 140 else 30 if a < 160 else 0

    # Left hip flexion
    if all(v is not None for v in [ls, lh, lk]):
        a = calculate_angle(ls, lh, lk)
        details['L_Hip'] = (lh, a)
        risks['L_Hip'] = 80 if a < 100 else 45 if a < 130 else 20 if a < 150 else 0

    # Right hip flexion
    if all(v is not None for v in [rs, rh, rk]):
        a = calculate_angle(rs, rh, rk)
        details['R_Hip'] = (rh, a)
        risks['R_Hip'] = 80 if a < 100 else 45 if a < 130 else 20 if a < 150 else 0

    # Trunk lean
    if all(v is not None for v in [ls, rs, lh, rh]):
        sm = ((ls[0]+rs[0])/2, (ls[1]+rs[1])/2)
        hm = ((lh[0]+rh[0])/2, (lh[1]+rh[1])/2)
        lean = trunk_lean_angle(sm, hm)
        details['Trunk'] = (tuple(map(int, sm)), lean)
        risks['Trunk'] = 75 if lean > 35 else 50 if lean > 25 else 25 if lean > 15 else 0

    # Shoulder asymmetry
    if all(v is not None for v in [ls, rs]):
        diff = abs(ls[1] - rs[1])
        risks['Shoulder_Asym'] = 70 if diff > 40 else 40 if diff > 25 else 20 if diff > 15 else 0

    # Hip asymmetry
    if all(v is not None for v in [lh, rh]):
        diff = abs(lh[1] - rh[1])
        risks['Hip_Asym'] = 70 if diff > 40 else 40 if diff > 25 else 20 if diff > 15 else 0

    # Ankle dorsiflexion
    for knee_kp, ankle_kp, key in [(lk, la, 'L_Ankle'), (rk, ra, 'R_Ankle')]:
        if all(v is not None for v in [knee_kp, ankle_kp]):
            vt = (ankle_kp[0] + (ankle_kp[0] - knee_kp[0]) * 0.3, ankle_kp[1] + 30)
            a  = calculate_angle(knee_kp, ankle_kp, vt)
            risks[key] = 60 if a > 120 else 30 if a > 110 else 0

    # Composite weighted score
    weights = {'L_ACL':0.25,'R_ACL':0.25,'L_Hip':0.10,'R_Hip':0.10,
               'Trunk':0.12,'Shoulder_Asym':0.05,'Hip_Asym':0.05,
               'L_Ankle':0.04,'R_Ankle':0.04}
    tw = sum(weights[k] for k in risks if k in weights)
    composite = int(sum(risks[k]*weights[k] for k in risks if k in weights) / tw) if tw > 0 else 0
    return risks, min(composite, 100), details


print('✅ Biomechanical engine ready')

✅ Biomechanical engine ready


## Cell 5 — Visualization & Alert Utilities

In [7]:
def risk_color(s):
    if s < 30:  return (0, 200, 0)      # Green
    if s < 60:  return (0, 200, 255)    # Yellow
    if s < 80:  return (0, 100, 255)    # Orange
    return (0, 0, 230)                  # Red

def risk_label(s):
    if s < 30:  return 'LOW'
    if s < 60:  return 'MODERATE'
    if s < 80:  return 'HIGH'
    return 'CRITICAL'


class AlertManager:
    def __init__(self):
        self.last_alert = 0
        self.flash_frames = 0

    def check(self, score):
        now = time.time()
        if score >= ALERT_THRESHOLD and (now - self.last_alert) > ALERT_COOLDOWN:
            self.last_alert = now
            self.flash_frames = 20
        if self.flash_frames > 0:
            self.flash_frames -= 1
            return True
        return False

    def draw(self, frame, score):
        h, w = frame.shape[:2]
        cv2.rectangle(frame, (0, 0), (w-1, h-1), (0, 0, 255), 8)
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, h//2-35), (w, h//2+35), (0, 0, 160), -1)
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
        msg = f'  INJURY RISK ALERT: {score}%  [{risk_label(score)}]  '
        cv2.putText(frame, msg, (w//2 - 300, h//2 + 12),
                    cv2.FONT_HERSHEY_DUPLEX, 0.95, (255, 255, 255), 2)
        return frame

alert_mgr = AlertManager()


def draw_dashboard(frame, composite, risks, person_idx=0, frame_num=0, fps=0.0):
    h, w = frame.shape[:2]
    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (280, 265), (12, 12, 12), -1)
    cv2.addWeighted(overlay, 0.65, frame, 0.35, 0, frame)

    cv2.putText(frame, f'INJURY RISK  P{person_idx+1}',
                (18, 34), cv2.FONT_HERSHEY_DUPLEX, 0.52, (200, 200, 200), 1)

    # Composite bar
    bx, by, bw, bh = 18, 42, 240, 20
    cv2.rectangle(frame, (bx, by), (bx+bw, by+bh), (55, 55, 55), -1)
    cv2.rectangle(frame, (bx, by), (bx+int(bw*composite/100), by+bh), risk_color(composite), -1)
    cv2.rectangle(frame, (bx, by), (bx+bw, by+bh), (110, 110, 110), 1)
    cv2.putText(frame, f'{risk_label(composite)}  {composite}%',
                (bx+4, by+15), cv2.FONT_HERSHEY_SIMPLEX, 0.48, (255,255,255), 1)

    labels = {'L_ACL':'L Knee (ACL)','R_ACL':'R Knee (ACL)',
              'L_Hip':'L Hip Flex','R_Hip':'R Hip Flex',
              'Trunk':'Trunk Lean','Shoulder_Asym':'Shldr Asym','Hip_Asym':'Hip Asym'}
    yo = 80
    for key, lbl in labels.items():
        val = risks.get(key)
        if val is None: continue
        cv2.putText(frame, lbl+':', (18, yo), cv2.FONT_HERSHEY_SIMPLEX, 0.37, (175,175,175), 1)
        mbw = 108
        cv2.rectangle(frame, (138, yo-10), (138+mbw, yo+3), (55,55,55), -1)
        cv2.rectangle(frame, (138, yo-10), (138+int(mbw*val/100), yo+3), risk_color(val), -1)
        cv2.putText(frame, str(val), (138+mbw+4, yo), cv2.FONT_HERSHEY_SIMPLEX, 0.37, (200,200,200), 1)
        yo += 20

    cv2.putText(frame, f'FPS:{fps:.1f}  F:{frame_num}',
                (w-140, h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.40, (140,140,140), 1)
    return frame


def draw_angle_on_joint(frame, jpos, angle, color):
    if jpos is None: return
    cv2.putText(frame, f'{angle:.0f}',
                (int(jpos[0])+8, int(jpos[1])-8),
                cv2.FONT_HERSHEY_SIMPLEX, 0.44, color, 1, cv2.LINE_AA)


def draw_risk_graph(frame, history, max_pts=80):
    h, w = frame.shape[:2]
    gw, gh = 190, 65
    gx, gy = w-gw-10, h-gh-28
    overlay = frame.copy()
    cv2.rectangle(overlay, (gx, gy), (gx+gw, gy+gh), (15,15,15), -1)
    cv2.addWeighted(overlay, 0.60, frame, 0.40, 0, frame)
    cv2.putText(frame, 'Risk History', (gx+4, gy-4), cv2.FONT_HERSHEY_SIMPLEX, 0.37, (150,150,150), 1)
    for pct, col in [(0.3,(0,200,0)),(0.6,(0,200,255)),(0.8,(0,0,230))]:
        ly = int(gy+gh-pct*gh)
        cv2.line(frame, (gx,ly), (gx+gw,ly), col, 1)
    recent = list(history)[-max_pts:]
    if len(recent) > 1:
        for i in range(1, len(recent)):
            x1 = int(gx+(i-1)/max_pts*gw); x2 = int(gx+i/max_pts*gw)
            y1 = int(gy+gh-recent[i-1]/100*gh); y2 = int(gy+gh-recent[i]/100*gh)
            cv2.line(frame, (x1,y1), (x2,y2), (255,255,255), 1, cv2.LINE_AA)
    return frame


def process_frame(frame, frame_count, fps):
    """Run YOLOv8 + biomechanics on one frame, return annotated frame + max risk."""
    results = model(frame, verbose=False)
    annotated = frame.copy()
    frame_max_risk = 0

    for r in results:
        if r.keypoints is None or r.keypoints.xy is None:
            continue
        annotated = r.plot(img=annotated, kpt_radius=4, line_width=2)
        keypoints = r.keypoints.xy.cpu().numpy()
        confs     = r.keypoints.conf.cpu().numpy()

        for p_idx, (pkp, pconf) in enumerate(zip(keypoints, confs)):
            risks, composite, details = analyze_person(pkp, pconf)
            frame_max_risk = max(frame_max_risk, composite)

            if SHOW_ANGLES:
                key_map = {'L_Knee':'L_ACL','R_Knee':'R_ACL',
                           'L_Hip':'L_Hip','R_Hip':'R_Hip','Trunk':'Trunk'}
                for jname, (jpos, angle) in details.items():
                    rk = key_map.get(jname, jname)
                    draw_angle_on_joint(annotated, jpos, angle, risk_color(risks.get(rk, 0)))

            annotated = draw_dashboard(annotated, composite, risks, p_idx, frame_count, fps)

    risk_history.append(frame_max_risk)
    all_risk_scores.append(frame_max_risk)

    if SHOW_GRAPH:
        annotated = draw_risk_graph(annotated, risk_history)

    if alert_mgr.check(frame_max_risk):
        annotated = alert_mgr.draw(annotated, frame_max_risk)

    return annotated, frame_max_risk


print('✅ Visualization & alert utilities ready')

✅ Visualization & alert utilities ready


## Cell 6 — 🎥 LIVE WEBCAM (VS Code)

- Opens a **live OpenCV window** with pose + risk overlay
- Press **`Q`** to quit and close the window
- Press **`S`** to save a snapshot
- Output video is saved automatically to `OUTPUT_PATH`

In [ ]:
all_risk_scores.clear()
risk_history.clear()
alert_mgr.flash_frames = 0

# ── Open webcam ───────────────────────────────────────────────────────────────
cap = cv2.VideoCapture(0)   # 0 = default webcam. Try 1 or 2 if wrong camera

if not cap.isOpened():
    print('❌ Cannot open webcam.')
    print('   Try changing VideoCapture(0) to VideoCapture(1)')
else:
    # Let camera warm up
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,  1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    time.sleep(1)

    w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30

    out_path = os.path.join(OUTPUT_FOLDER, 'webcam_output.mp4')
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    print(f'🎥 Webcam started  [{w}x{h} @ {fps:.0f}fps]')
    print(f'   Saving to: {out_path}')
    print('   Press  Q  to quit | Press  S  to save snapshot')

    frame_count = 0
    fps_times   = deque(maxlen=30)
    snap_count  = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print('⚠️  Lost webcam feed')
            break

        frame_count += 1

        # Skip frames for performance
        if frame_count % FRAME_SKIP != 0:
            cv2.imshow('Injury Risk Monitor  [Q=quit  S=snapshot]', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue

        # Compute live FPS
        fps_times.append(time.time())
        live_fps = (len(fps_times)/(fps_times[-1]-fps_times[0]+1e-6)
                    if len(fps_times) > 1 else 0)

        annotated, risk = process_frame(frame, frame_count, live_fps)
        out.write(annotated)

        # ── Show in window ──────────────────────────────────────────────────
        cv2.imshow('Injury Risk Monitor  [Q=quit  S=snapshot]', annotated)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            snap_count += 1
            snap_path = os.path.join(OUTPUT_FOLDER, f'snapshot_{snap_count}.jpg')
            cv2.imwrite(snap_path, annotated)
            print(f'📸 Snapshot saved: {snap_path}')

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f'\n✅ Webcam session ended')
    print(f'   Frames recorded : {frame_count}')
    if all_risk_scores:
        print(f'   Average risk    : {np.mean(all_risk_scores):.1f}%')
        print(f'   Peak risk       : {max(all_risk_scores)}%')
    print(f'   Video saved to  : {out_path}')

🎥 Webcam started  [1280x720 @ 30fps]
   Saving to: /Users/namantulsyan/Desktop/injury_risk_outputs/webcam_output.mp4
   Press  Q  to quit | Press  S  to save snapshot


## Cell 7 — 📁 Process a Video File
Set `VIDEO_PATH` in Cell 2, then run this cell.

In [ ]:
all_risk_scores.clear()
risk_history.clear()
alert_mgr.flash_frames = 0

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise FileNotFoundError(f'❌ Cannot open video: {VIDEO_PATH}')

w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30
total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out_path = os.path.join(OUTPUT_FOLDER, 'video_output.mp4')
out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

print(f'📹 Processing: {VIDEO_PATH}')
print(f'   Resolution : {w}x{h} @ {fps:.0f}fps  ({total} frames total)')
print(f'   Max frames : {MAX_FRAMES or "all"}')
print(f'   Output     : {out_path}')
print(f'   Press Q in the preview window to stop early\n')

frame_count = 0
fps_times   = deque(maxlen=30)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if MAX_FRAMES and frame_count > MAX_FRAMES:
        break
    if frame_count % FRAME_SKIP != 0:
        continue

    fps_times.append(time.time())
    live_fps = (len(fps_times)/(fps_times[-1]-fps_times[0]+1e-6)
                if len(fps_times) > 1 else 0)

    annotated, risk = process_frame(frame, frame_count, live_fps)
    out.write(annotated)

    # Live preview window
    cv2.imshow('Injury Risk — Video  [Q=quit]', annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print('⏹  Stopped by user')
        break

    if frame_count % 50 == 0:
        avg = np.mean(all_risk_scores) if all_risk_scores else 0
        print(f'  Frame {frame_count:4d}/{total}  |  Avg: {avg:.1f}%  |  Current: {risk}%  |  FPS: {live_fps:.1f}')

cap.release()
out.release()
cv2.destroyAllWindows()

print(f'\n✅ Done! {frame_count} frames processed → {out_path}')

## Cell 8 — 🌐 Streamlit Web Dashboard

Writes `app_realtime.py` and launches it. Opens automatically at `http://localhost:8501`

In [ ]:
streamlit_code = '''
import streamlit as st
import cv2, numpy as np, math, time, os, tempfile
from collections import deque
from ultralytics import YOLO

st.set_page_config(page_title="Injury Risk Monitor", layout="wide", page_icon="🏃")
st.markdown("""
<style>
body, .main { background:#0f0f1a; color:white; }
.stMetric { background:#1a1a2e; border-radius:8px; padding:10px; }
</style>
""", unsafe_allow_html=True)

st.title("🏃 Real-Time Injury Risk Prediction")

with st.sidebar:
    st.header("⚙️ Settings")
    model_size   = st.selectbox("Model", ["n","s","m"], index=0)
    conf_thresh  = st.slider("Confidence", 0.1, 0.9, 0.5, 0.05)
    frame_skip   = st.slider("Frame Skip", 1, 5, 1)
    alert_thresh = st.slider("Alert Threshold (%)", 30, 90, 70)

@st.cache_resource
def load_model(s):
    return YOLO(f"yolov8{s}-pose.pt")
model = load_model(model_size)

def calculate_angle(a, b, c):
    a,b,c = np.array(a,float),np.array(b,float),np.array(c,float)
    ba,bc = a-b,c-b
    n1,n2 = np.linalg.norm(ba),np.linalg.norm(bc)
    if n1==0 or n2==0: return 180.0
    return float(np.degrees(np.arccos(np.clip(np.dot(ba,bc)/(n1*n2),-1,1))))

def risk_label(s): return "LOW" if s<30 else "MODERATE" if s<60 else "HIGH" if s<80 else "CRITICAL"
def risk_hex(s): return "#27ae60" if s<30 else "#f39c12" if s<60 else "#e67e22" if s<80 else "#e74c3c"

def analyze(frame):
    results = model(frame, verbose=False, conf=conf_thresh)
    annotated = frame.copy()
    max_risk, max_risks = 0, {}
    for r in results:
        if r.keypoints is None: continue
        annotated = r.plot(img=annotated, kpt_radius=4, line_width=2)
        kpts  = r.keypoints.xy.cpu().numpy()
        confs = r.keypoints.conf.cpu().numpy()
        for pkp, pconf in zip(kpts, confs):
            def kp(i):
                return pkp[i] if pconf[i]>conf_thresh and pkp[i][0]>0 and pkp[i][1]>0 else None
            ls,rs=kp(5),kp(6); lh,rh=kp(11),kp(12); lk,rk=kp(13),kp(14); la,ra=kp(15),kp(16)
            risks={}
            if all(v is not None for v in [lh,lk,la]):
                a=calculate_angle(lh,lk,la); risks["L_ACL"]=90 if a<120 else 60 if a<140 else 30 if a<160 else 0
            if all(v is not None for v in [rh,rk,ra]):
                a=calculate_angle(rh,rk,ra); risks["R_ACL"]=90 if a<120 else 60 if a<140 else 30 if a<160 else 0
            if all(v is not None for v in [ls,lh,lk]):
                a=calculate_angle(ls,lh,lk); risks["L_Hip"]=80 if a<100 else 45 if a<130 else 20 if a<150 else 0
            if all(v is not None for v in [rs,rh,rk]):
                a=calculate_angle(rs,rh,rk); risks["R_Hip"]=80 if a<100 else 45 if a<130 else 20 if a<150 else 0
            if all(v is not None for v in [ls,rs,lh,rh]):
                sm=((ls[0]+rs[0])/2,(ls[1]+rs[1])/2); hm=((lh[0]+rh[0])/2,(lh[1]+rh[1])/2)
                dx,dy=hm[0]-sm[0],hm[1]-sm[1]
                lean=abs(math.degrees(math.atan2(dx,dy))) if dy!=0 else 0
                risks["Trunk"]=75 if lean>35 else 50 if lean>25 else 25 if lean>15 else 0
            weights={"L_ACL":0.25,"R_ACL":0.25,"L_Hip":0.10,"R_Hip":0.10,"Trunk":0.12}
            tw=sum(weights[k] for k in risks if k in weights)
            comp=int(sum(risks[k]*weights[k] for k in risks if k in weights)/tw) if tw>0 else 0
            comp=min(comp,100)
            col=(0,200,0) if comp<30 else (0,200,255) if comp<60 else (0,100,255) if comp<80 else (0,0,230)
            cv2.putText(annotated,f"Risk: {comp}% [{risk_label(comp)}]",(15,40),cv2.FONT_HERSHEY_DUPLEX,1.1,col,2)
            if comp>=alert_thresh:
                fh,fw=annotated.shape[:2]; cv2.rectangle(annotated,(0,0),(fw-1,fh-1),(0,0,255),8)
            if comp>max_risk: max_risk,max_risks=comp,risks
    return annotated, max_risk, max_risks

uploaded = st.file_uploader("Upload a video", type=["mp4","mov","avi"])
col1, col2 = st.columns([3,1])

if uploaded:
    tfile = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    tfile.write(uploaded.read()); tfile.close()
    cap = cv2.VideoCapture(tfile.name)
    frame_ph  = col1.empty()
    score_ph  = col2.empty()
    detail_ph = col2.empty()
    all_scores = []; fc = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        fc+=1
        if fc%frame_skip!=0: continue
        annotated, comp, risks = analyze(frame)
        all_scores.append(comp)
        frame_ph.image(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB), use_column_width=True)
        c=risk_hex(comp)
        score_ph.markdown(f"""
        <div style="background:#1a1a2e;border-radius:12px;padding:20px;text-align:center;">
            <div style="font-size:52px;font-weight:bold;color:{c}">{comp}%</div>
            <div style="font-size:18px;color:{c}">{risk_label(comp)}</div>
            <div style="color:#888;font-size:12px;">Frame {fc} | Avg: {np.mean(all_scores):.1f}%</div>
        </div>""", unsafe_allow_html=True)
        factor_labels={"L_ACL":"L Knee","R_ACL":"R Knee","L_Hip":"L Hip","R_Hip":"R Hip","Trunk":"Trunk"}
        md=""
        for k,lb in factor_labels.items():
            v=risks.get(k,0); c2=risk_hex(v)
            md+=f"<div style=\'margin:5px 0\'><span style=\'color:#aaa\'>{lb}:</span> <span style=\'color:{c2}\'>{v}%</span>"
            md+=f"<div style=\'background:#333;border-radius:4px;height:7px;margin:2px 0\'><div style=\'background:{c2};width:{v}%;height:7px;border-radius:4px\'></div></div></div>"
        detail_ph.markdown(md, unsafe_allow_html=True)
    cap.release(); os.unlink(tfile.name)
    st.success(f"Done! {fc} frames | Peak: {max(all_scores) if all_scores else 0}%")
'''

with open('app_realtime.py', 'w') as f:
    f.write(streamlit_code)

import subprocess, time, webbrowser

proc = subprocess.Popen(
    ['streamlit', 'run', 'app_realtime.py',
     '--server.port', '8501',
     '--server.headless', 'true'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(4)
webbrowser.open('http://localhost:8501')
print('🌐 Streamlit app launched!')
print('   Opening: http://localhost:8501')
print('   (Upload a video in the browser to analyze it)')

## Cell 9 — 📊 Summary Report
Run after Cell 6 or Cell 7 to see the full risk analysis.

In [ ]:
if not all_risk_scores:
    print('⚠️  No scores yet — run Cell 6 (webcam) or Cell 7 (video) first')
else:
    scores = np.array(all_risk_scores)

    print('═' * 48)
    print('  📊  INJURY RISK ANALYSIS SUMMARY')
    print('═' * 48)
    print(f'  Total frames analyzed : {len(scores)}')
    print(f'  Average risk score    : {scores.mean():.1f}%')
    print(f'  Peak risk score       : {scores.max():.0f}%')
    print(f'  Frames LOW  (<30%)    : {(scores<30).sum():4d}  ({100*(scores<30).mean():.0f}%)')
    print(f'  Frames MOD  (30-60%)  : {((scores>=30)&(scores<60)).sum():4d}  ({100*((scores>=30)&(scores<60)).mean():.0f}%)')
    print(f'  Frames HIGH (60-80%)  : {((scores>=60)&(scores<80)).sum():4d}  ({100*((scores>=60)&(scores<80)).mean():.0f}%)')
    print(f'  Frames CRIT (>80%)    : {(scores>=80).sum():4d}  ({100*(scores>=80).mean():.0f}%)')
    print('═' * 48)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
    fig.patch.set_facecolor('#1a1a2e')

    ax1.set_facecolor('#16213e')
    ax1.plot(scores, color='#e94560', linewidth=0.8, alpha=0.9, label='Risk')
    ax1.fill_between(range(len(scores)), scores, alpha=0.15, color='#e94560')
    ax1.axhline(30, color='#00c853', ls='--', lw=1, alpha=0.7, label='Low/Mod')
    ax1.axhline(60, color='#ffd600', ls='--', lw=1, alpha=0.7, label='Mod/High')
    ax1.axhline(80, color='#dd2c00', ls='--', lw=1, alpha=0.7, label='High/Crit')
    ax1.axhline(ALERT_THRESHOLD, color='#ff6d00', ls=':', lw=1.5, label=f'Alert ({ALERT_THRESHOLD}%)')
    ax1.set_xlim(0, len(scores)); ax1.set_ylim(0, 100)
    ax1.set_xlabel('Frame', color='white'); ax1.set_ylabel('Risk Score (%)', color='white')
    ax1.set_title('Injury Risk Over Time', color='white', fontsize=13)
    ax1.tick_params(colors='white')
    ax1.legend(fontsize=8, facecolor='#1a1a2e', labelcolor='white')
    for sp in ax1.spines.values(): sp.set_color('#444')

    ax2.set_facecolor('#16213e')
    cats   = ['LOW\n(<30%)', 'MOD\n(30-60%)', 'HIGH\n(60-80%)', 'CRIT\n(>80%)']
    counts = [(scores<30).sum(),((scores>=30)&(scores<60)).sum(),
               ((scores>=60)&(scores<80)).sum(),(scores>=80).sum()]
    bars = ax2.bar(cats, counts, color=['#00c853','#ffd600','#ff6d00','#dd2c00'],
                   edgecolor='#444', linewidth=0.8)
    for bar, cnt in zip(bars, counts):
        if cnt > 0:
            ax2.text(bar.get_x()+bar.get_width()/2, bar.get_height()+1,
                     str(cnt), ha='center', color='white', fontsize=10)
    ax2.set_ylabel('Frame Count', color='white')
    ax2.set_title('Risk Level Distribution', color='white', fontsize=13)
    ax2.tick_params(colors='white')
    for sp in ax2.spines.values(): sp.set_color('#444')

    plt.tight_layout()
    chart_path = os.path.join(OUTPUT_FOLDER, 'risk_summary.png')
    plt.savefig(chart_path, dpi=120, bbox_inches='tight', facecolor='#1a1a2e')
    plt.show()
    print(f'\n✅ Chart saved: {chart_path}')

## Cell 10 — 💾 Save All Output Files
Copies everything to your Desktop `injury_risk_outputs` folder.

In [ ]:
import shutil, os

files_to_save = [
    ('webcam_output.mp4',  os.path.join(OUTPUT_FOLDER, 'webcam_output.mp4')),
    ('video_output.mp4',   os.path.join(OUTPUT_FOLDER, 'video_output.mp4')),
    ('risk_summary.png',   os.path.join(OUTPUT_FOLDER, 'risk_summary.png')),
    ('app_realtime.py',    os.path.join(OUTPUT_FOLDER, 'app_realtime.py')),
]

print(f'📁 Output folder: {OUTPUT_FOLDER}\n')
for src, dst in files_to_save:
    # src might already be in output folder
    if os.path.exists(src) and src != dst:
        shutil.copy(src, dst)
        print(f'✅ Saved: {dst}')
    elif os.path.exists(dst):
        print(f'✅ Already in folder: {os.path.basename(dst)}')
    else:
        print(f'⚠️  Not found yet: {os.path.basename(src)}  (run earlier cells first)')

print('\n🎉 Done! Open the folder to access your files.')

# Auto-open the output folder
import platform, subprocess
if platform.system() == 'Windows':
    os.startfile(OUTPUT_FOLDER)
elif platform.system() == 'Darwin':  # macOS
    subprocess.Popen(['open', OUTPUT_FOLDER])
else:  # Linux
    subprocess.Popen(['xdg-open', OUTPUT_FOLDER])